This notebook extracts the features of the legitimate URLs in the legitimate.csv file. The extracted features are then stored in the legitimate_extracted.csv file. 

In [ ]:
import pandas as pd

In [ ]:
# importing required packages for this section
from urllib.parse import urlparse, urlencode
import ipaddress
import re
import requests

In [ ]:
%pip install python-whois


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 KB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-whois: filename=python_whois-0.8.0-py3-none-any.whl size=103262 sha256=51c1883228ecad1a91d2fc0fe5ab70f52f767e40287648d25b7e452023c5645f
  Stored in directory: /root/.cache/pip/wheels/24/20/6b/5550a3d6bef09ddaed74eb84006fc3d53f94867f1916794df4
Successfully built python-whois


In [ ]:
import re
from bs4 import BeautifulSoup
import whois
import urllib
import urllib.request
from datetime import datetime

Domain Name Finder

In [ ]:
def getDomain(url):
  domain = urlparse(url).netloc
  if re.match(r"^www.", domain):
      domain = domain.replace("www.", "")
  return domain

1.Presence of Special Symbols

In [ ]:
def special_symbols(url):
    if '@' in url:
        return 1
    else:
        return 0

2.Presence of sub-domains

In [ ]:
def check_subdomain(url):
    count = 0
    for i in url:
        if i == '.':
            count+=1
    if count<=3:
        return 0
    else:
        return 1


3.looking for '-' in domain

In [ ]:
def prefixSuffix(url):
    domain = urlparse(url).netloc
    if '-' in domain or '_' in domain:
        return 1            # phishing
    else:
        return 0


4.Using IP instead of URL

In [ ]:
def havingIP(url):
    try:
        ipaddress.ip_address(url)
        ip = 1
    except:
        ip=0
    return ip

5.Depth of URL

In [ ]:
def getDepth(url):
  s = urlparse(url).path.split('/')
  depth = 0
  for j in range(len(s)):
    if len(s[j]) != 0:
      depth = depth+1
  return depth


6.Redirections in URL

In [ ]:
def redirect(url):
    pos = url.rfind("//")
    if pos>7:
        return 1
    else:
        return 0


7.URL shortening services

In [ ]:
#listing shortening services
shortening_services = r"bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|" \
                      r"yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|" \
                      r"short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|" \
                      r"doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|db\.tt|" \
                      r"qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|q\.gs|is\.gd|" \
                      r"po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|x\.co|" \
                      r"prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|" \
                      r"tr\.im|link\.zip\.net"


In [ ]:
def tinyURL(url):
    match = re.search(shortening_services, url)
    if match:
        return 1
    else:
        return 0

8.Number of digits in URL

In [ ]:
def DigitCount(url):
  count=0
  for i in url:
    if i.isdigit():
      count+=1
  return count

9.Web Traffic

In [ ]:
def web_traffic(url_in):
  try:
    if 'http' not in url_in and 'https' not in url_in:
      url_in='http://'+url_in

    domain_name = whois.whois(urlparse(url_in).netloc).domain_name
    if(type(domain_name) is list ):
      domain_name=domain_name[1].lower()
    if domain_name.isupper():
      domain_name=domain_name.lower()
    r = requests.get('https://siterankdata.com/'+domain_name)
    soup = BeautifulSoup(r.text,'html.parser')
    res = str(soup.find_all('meta')[3])

    a = res.split()[7].split('.')[0]
    rank=int(a)
  except:
      return 1
  if rank > 1000000:
    return 1
  else:
    return 0


10.Domain age

In [ ]:
def domainAge(domain_name):
  creation_date = domain_name.creation_date
  expiration_date = domain_name.expiration_date
  if (isinstance(creation_date, str) or isinstance(expiration_date, str)):
    try:
      creation_date = datetime.strptime(creation_date, "%Y-%m-%d")
      expiration_date = datetime.strptime(expiration_date, "%Y-%m-%d")
    except:
      return 1
  if ((expiration_date is None) or (creation_date is None)):
      return 1
  elif ((type(expiration_date) is list) or (type(creation_date) is list)):
    try:
      ageofdomain = abs((expiration_date[0]-creation_date[0]).days)
    except:
      return 1
  else:
    ageofdomain = abs((expiration_date - creation_date).days)
  if ((ageofdomain/30) < 6):
    age = 1
  else:
    age = 0
  return age


Processing the URL's

In [ ]:
#Function to extract features
def featureExtraction(url,label):
  if 'http' not in url and 'https' not in url:
    url='http://'+url
  v = web_traffic(url)
  dns = 0
  try:
    domain_name = whois.whois(urlparse(url).netloc)
  except:
    dns = 1
  
  features = []
  #Address bar based features (11)
  features.append(getDomain(url)) 
  features.append(v)
  features.append(special_symbols(url))
  features.append(check_subdomain(url))
  features.append(prefixSuffix(url))
  features.append(havingIP(url))
  features.append(getDepth(url))
  features.append(redirect(url))
  features.append(tinyURL(url))
  features.append(DigitCount(url))
  features.append(1 if dns == 1 else domainAge(domain_name))
  features.append(label)
  
  return features

Legit URLS

In [18]:
#Extracting the feautres & storing them in a list
legiurl = pd.read_csv('legitimate.csv')
legi_features = []
label = 0

for i in range(0,10000):
  url = legiurl['url'][i]
  legi_features.append(featureExtraction(url, label))

4862
4863
4864
4865
4866
4867
4868
4869
4870
4871
4872
4873
4874
Error trying to connect to socket: closing socket - [Errno -2] Name or service not known
Error trying to connect to socket: closing socket - [Errno -2] Name or service not known
4875
4876
4877
4878
4879
4880
4881
4882
4883
4884
4885
4886
4887
4888
4889
4890
4891
4892
4893
4894
4895
4896
4897
4898
4899
4900
4901
4902
4903
4904
4905
4906
4907
4908
4909
4910
4911
4912
4913
4914
4915
4916
4917
4918
4919
4920
4921
4922
4923
4924
4925
4926
4927
4928
4929
4930
4931
4932
4933
4934
4935
4936
4937
4938
4939
4940
4941
4942
4943
4944
4945
4946
4947
4948
4949
4950
4951
4952
4953
4954
4955
4956
4957
4958
4959
4960
4961
4962
4963
4964
4965
4966
4967
4968
4969
4970
4971
4972
4973
4974
4975
4976
4977
4978
4979
4980
4982
4983
4984
Error trying to connect to socket: closing socket - timed out
4985
4986
4987
4988
4989
4990
4991
4992
4993
4994
4995
4996
4997
4998
4999


In [19]:
#converting the list to dataframe
feature_names = ['Domain', 'Web_Traffic', 'Have_At', 'SubDomain','Prefix/Suffix', 'Have_IP', 'URL_Depth', 'Redirection',
                 'TinyURL', 'Digit_count','Domain_Age', 'Label']

legitimate = pd.DataFrame(legi_features, columns=feature_names)
legitimate.head()

,Domain,Web_Traffic,Have_At,SubDomain,Prefix/Suffix,Have_IP,URL_Depth,Redirection,TinyURL,Digit_count,Domain_Age,Label
0,en.turkeyforfriends.com,0,0,0,0,0,2,0,0,6,0,0
1,blogs.indiewire.com,0,0,0,0,0,3,0,0,0,0,0
2,absoluteastronomy.com,0,0,0,0,0,2,0,0,0,0,0
3,flickr.com,0,0,0,0,0,3,0,0,10,0,0
4,olx.co.id,0,0,0,0,0,2,0,1,5,0,0


In [20]:
legitimate.to_csv('legitimate_extracted.csv', index=False)